# NIDLookUP (USE ilearn2.0)
*   程式簡介
> 校內目前並未提供任何可以查尋 NID 的方式，本次利用 ilearn 提供之簡易查詢功能（NID 查詢姓名），實作出一個可以暴破計算出欲查詢者 NID 的腳本（反推爆破 使姓名可以返查 NID）

*   主要查詢
> 目前查詢功能的實作已經可以因應各種模式，並透過不同的參數引入，可查詢更多的 NID 資料


# Import


In [0]:
from bs4 import BeautifulSoup
import requests, getpass, csv, re

# Set Session

In [0]:
s = requests.Session()

# Set login Data

In [0]:
username = input('請輸入學號:')
password = getpass.getpass('請輸入密碼:')

# ilearn Tool

## 功能 00：Login ilearn

In [0]:
# Set login url
loginPage_url = 'https://ilearn2.fcu.edu.tw/login/index.php'

# Get the logintoken
loginPage = s.get(loginPage_url)
bs_loginPage = BeautifulSoup(loginPage.text, 'html.parser')

login_data = {
    'username': username,
    'password': password,
    'logintoken' : bs_loginPage.find_all(attrs={"name": "logintoken"})[0]['value']
}

# login
try:
    loginPage = s.post(loginPage_url, data = login_data)
    bs_loginPage = BeautifulSoup(loginPage.text, 'html.parser')

    # print(loginPage.text)
    print('%s 登入成功' %(bs_loginPage.find_all(attrs={"class":"userpicture defaultuserpic"})[0]['alt'][0:3]))

except:
    print('登入失敗')

## 功能 01：查詢所有授課老師
*   ilearn ID
*   課程名稱
*   老師名稱





In [0]:
# define course
class course():
    def __init__(self, name, ID):
        self.name = name
        self.ID = ID
        self.teacher = []
    def add_teachet(self, teacher):
        self.teacher.append(teacher)

# All_course_list
All_course = []

# Set url
url = 'https://ilearn2.fcu.edu.tw/'
page = s.get(url)
bs_page = BeautifulSoup(page.text, 'html.parser')

# 查詢課表
course_list = bs_loginPage.find_all(attrs={"class":"coc-mycurricular coursebox clearfix"})

# print(len(course_list))

for i in range(len(course_list)):
    try:
        course_name = course_list[i].find_all('a')[0]['title']
        
        course_id = course_list[i].find_all('a')[0]['href'][46:51]

        temp = course(course_name, course_id)

        try:
            course_teacher = course_list[i].find_all('span')
            for j in range(len(course_teacher)):
                if j == 0:
                    course_teacher_name = course_teacher[j].text[4:]
                    temp.add_teachet(course_teacher_name)
                else:
                    course_teacher_name = course_teacher[j].text[1:]
                    temp.add_teachet(course_teacher_name)
        except:
            print()

        All_course.append(temp)

    except:
        print('Error')

In [0]:
for i in All_course:
    print('ID：' + i.ID)
    print('URL：'+ 'https://ilearn2.fcu.edu.tw/user/index.php?id=' + i.ID)
    print('課程名稱：' + i.name)
    print('老師：' + str(i.teacher))
    print()

## 功能02：查詢 NID

In [0]:
def SearchNID(Course_ID, status, name):
    # Set NID & StudentMode
    NID = status
    StudentMode = True

    # Set URL
    url = ('https://ilearn2.fcu.edu.tw/user/index.php?id=' + str(Course_ID))
    page = s.get(url)
    bs_page = BeautifulSoup(page.text, 'html.parser')

    # Get contextid
    contextid = bs_page.find_all('body')[0]['class'][11][8:]

    # Get perpage（課程參加人數）
    perpage = bs_page.find_all(attrs={"class": "userlist"})[0].find('p').text[5:]

    # Fix URL
    url = ('https://ilearn2.fcu.edu.tw/user/index.php?contextid='+ contextid +'&id=' + Course_ID + '&perpage=' + perpage)

    # Check StudentMode
    page = s.get(url)
    bs_page = BeautifulSoup(page.text, 'html.parser')

    try:
        if bs_page.find_all(attrs={"value": "加入用戶到此課程"})[0]['value'] == '加入用戶到此課程':
            StudentMode = False
            MemberList = bs_page.find_all('tbody')[0]
        for i in range(int(perpage)):
            Member_name = MemberList.find_all(attrs={"id":"user-index-participants-" + Course_ID + "_r"+ str(i) +"_c1"})[0].find('img')['alt'][0:3]
            Member_NID = MemberList.find_all(attrs={"id":"user-index-participants-" + Course_ID + "_r" + str(i) + "_c2"})[0].text
            Member_status = MemberList.find_all(attrs={"id":"user-index-participants-" + Course_ID + "_r" + str(i) + "_c3"})[0].text[14:-15]
            
            if Member_status == '助理教師':
                Member_status = '助教'
            
            if Member_name == name:
                return Member_NID
        
            if name == 'ALL':
                print ('%3s %-8s %s' %(Member_name, Member_NID, Member_status))
    except:
        if name == 'ALL' and StudentMode == True:
            return '權限不足'

    # 檢查是否有此人
    data = {
        'unified-filters[]': name,
        'unified-filter-submitted': '1'
    }

    page = s.post(url, data = data)
    bs_page = BeautifulSoup(page.text, 'html.parser')

    try:
        check = bs_page.find_all(attrs={"id": "user-index-participants-" + Course_ID + "_r0_c0"})[0].find('img')['alt'][0:3]
        perpage = bs_page.find_all(attrs={"class": "userlist"})[0].find('p').text[5:]
        (name == check and perpage == '1') == True
    except:
        return ('查無此人：')

    searchMode = 0

    # 搜尋模式：老師
    if status == 'T':
        searchMode = 6
    elif status == 'D':
        searchMode = 8
    
    while len(NID) != searchMode:
        for i in range(0, 10):
            # print(NID)
            data = {
                'unified-filters[]': NID + str(i),
                'unified-filter-submitted': '1'
            }

            page = s.post(url, data = data)
            bs_page = BeautifulSoup(page.text, 'html.parser')
            
            
            perpage = bs_page.find_all(attrs={"class": "userlist"})[0].find('p').text[5:]

            try:
                MemberList = bs_page.find_all('tbody')[0]
                
                Still_in_list = False

                for j in range(int(perpage)):
                    Member_name = MemberList.find_all(attrs={"id":"user-index-participants-" + Course_ID + "_r" + str(j) +"_c0"})[0].find('img')['alt'][0:3]
                    Member_status = MemberList.find_all(attrs={"id":"user-index-participants-"+ Course_ID + "_r" + str(j) + "_c1"})[0].text
                    if name == Member_name:
                        Still_in_list = True
                        break

                if Still_in_list == True:
                    NID += str(i)
                else:
                    continue
            except:
                continue
    return NID

## 功能03：顯示所有老師證號 與 輸出 CSV

In [0]:
with open('outputCSV-NID.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['ID', '課程', '老師姓名 證號'])

    for i in All_course:
        print('ID：' + i.ID)
        print('課程：' + i.name)
        print('老師：', end='')

        if i.teacher == None:
            print('\n')
        
        tempstr = ''

        for j in i.teacher:
            tempNID = SearchNID(i.ID, 'T', j)
            
            print(j, tempNID ,end=' ')

            tempstr = tempstr + j + ' ' + tempNID + ' '
        
        writer.writerow([i.ID, i.name, tempstr])

        print('\n')

## 功能04：輸出 CVS 檔案

In [0]:
with open('output.csv', 'w', newline='') as csvfile:
    # 建立 CSV 檔寫入器
    writer = csv.writer(csvfile)

    writer.writerow(['ID', '課程', '老師姓名 證號'])
    
    for i in All_course:
        

        for j in i.teacher:
            tempstr = tempstr + str(SearchNID(i.ID, 'T', j))
            print(j, SearchNID(i.ID, 'T', j) ,end=' ')


        writer.write
        print('ID：' + i.ID)
        print('課程：' + i.name)
        print('老師：', end='')
        
        
        print('\n')

